In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
from statsmodels.stats.weightstats import ttest_ind
import cem
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import scipy.stats as stats

### Read in Data

In [ ]:
datapath = '/Users/jm7581/Documents/projects/active/AITA/data/'
resultspath = '/Users/jm7581/Documents/projects/active/AITA/results/adroit/'
graphpath = '/Users/jm7581/Documents/projects/active/AITA/graphs/'

In [ ]:
data = pd.read_stata('/Users/jm7581/Documents/projects/active/AITA/data/aita_data_2022_prepped.dta')

#filter sample
data['num_comments'] = data.groupby('postid')['postid'].transform('count')
data = data[data['cum_score_absolute'] != 0]
data['num_comments'] = data.groupby('postid')['postid'].transform('count')
data = data[data['num_comments'] > 15]

### Perform Matching

In [ ]:
O = 'score_log'
T = 'consensus_disagree'

Y = data['score_log']

#to exact match - include variable here, but not in schema
X = data[['consensus_disagree', 'cs_majority', 'time_diff_min_ln', 
          'comment_competition_ln', 'author_skill_ln', 'comment_length_ln']]



schema = {
   'cs_majority': (pd.qcut, {'q':12}),
    'time_diff_min_ln': (pd.qcut, {'q':12}),  
    'comment_competition_ln': (pd.qcut, {'q':25}), 
       'author_skill_ln': (pd.qcut, {'q':12}),    
       'comment_length_ln': (pd.qcut, {'q':12}),    
   }


X_coarse = X.apply(lambda x: schema[x.name][0](x, **schema[x.name][1]) if x.name in schema else x)


# Perform CEM matching
%time weights = match(X_coarse, T)

data['match_weight'] = weights


In [ ]:
data.to_stata('/Users/jm7581/Documents/Projects/Active/AITA/data/aita_data_2022_prepped_matched.dta')